In [ ]:
!pip install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 3.0 MB/s eta 0:00:00


Upload files:

In [22]:
import pandas as pd
from IPython.display import display
import os
import matplotlib.pyplot as plt
import math
import numpy as np
from nltk.tokenize import sent_tokenize
import string
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold, RandomizedSearchCV, RepeatedKFold
from pandas.plotting import scatter_matrix
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.naive_bayes import GaussianNB, MultinomialNB, CategoricalNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.metrics import accuracy_score, hamming_loss, mean_squared_error

encoder=OneHotEncoder()

class DataFrameSelector(BaseEstimator):
    
    def __init__(self, attribute_names):
        self.attribute_names= attribute_names
        
    def fit(self,X, y = None):
        return self
    
    def transform(self, X):
        return X[self.attribute_names].values


local_path = '/content/'

# files = ['bahai', 'buddhism', 'christianity', 'hinduism', 'historical', 'islam','judaism', 'p_j_s', 'shinto', 't_c', 'iranian']
files = ['buddhism', 'christianity', 'hinduism', 'islam','judaism', 'p_j_s', 'shinto', 't_c']
additions = ['BU', 'CH', 'AS',  'IS', 'JU', 'AS', 'EA', 'EA']
frames = [0]*8
results = [0]*8

def addColumn(files, additions, frames):
    for i in range(len(files)):
        csv_path=os.path.join(local_path, files[i] + '.csv')
        # results[i] = pd.read_csv(csv_path, nrows=4500)
        # print(len(results[i]))
        df = pd.read_csv(csv_path, nrows=4500) 
        df2 = df.assign(Religion=additions[i])
        frames[i] = df2

# # combine dataframes to one
addColumn(files, additions, frames)
combined_df = pd.concat(frames, ignore_index=True)
combined_df = combined_df.rename(columns={"Language of Original Document": "Language"})
combined_df = combined_df.rename(columns={"Funding Details": "Funding"})
combined_df = combined_df.rename(columns={"Cited by": "Cited"})

Imputation:

In [11]:
# # IMPUTATION
def imputation(combined_df):
    north = ["Hong", "Macau", "Israel", "Japan", "Singapore", "Korea", "Canada", "Europe", "Russia", "Taiwan", "Australia", "Zealand", "Taiwan", "US", "UK", "Germany", "France", "Spain", "Netherlands", "Italy", "Cyprus", "Austria", "Belgium", "Bulgaria", "Croatia", "Czech", "Denmark", "Estonia", "Finland", "Greece", "Hungary", "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia", "Sweden", "Iceland", "Liechtenstein", "Norway", "States", "Kingdom", "Toronto", "Switzerland"]
    # western european languages that spread as  a result of colonisation
    lang_powerhouses = ["English", "French", "Spanish", "Portuguese", "German", "Afrikaans" ]


#     # replace funding details with 0 if nan, 1 else
    combined_df["Funding"].where(combined_df["Funding"] != combined_df["Funding"], 1, inplace = True)
    combined_df["Funding"].where(combined_df["Funding"] == 1, 0, inplace = True)


#     # drop column link & column year
    combined_df.drop(columns=["Link"], axis=1, inplace=True)
    combined_df.drop(columns=["Year"], axis=1, inplace=True)


#     # replace nominal categories (RE) to numerical -> 
    combined_df = pd.get_dummies(combined_df, columns=["Religion"])

#     # remove rows where affilitations is null
    combined_df = combined_df.dropna(subset=["Affiliations"])
    combined_df.reset_index(drop=True, inplace=True)
#     print(len(combined_df))


#     # remove punctuation
    for i in range(len(combined_df)-1):
        combined_df.loc[i+1, "Affiliations"] = combined_df.loc[i+1, "Affiliations"].translate(str.maketrans('', '', string.punctuation))

#   # if affiliation in north, replace with 0
    for i in range (len(north)):
        combined_df.Affiliations = combined_df.Affiliations.apply(lambda x: '0' if north[i] in x else x)
       

#     # if affiliation not in north, replace with 1
    filter  = (combined_df['Affiliations'] != '0' )
    combined_df.loc[filter, 'Affiliations'] = 1

#     # 0 value from string to int
    filter  = (combined_df['Affiliations'] == '0' )
    combined_df.loc[filter, 'Affiliations'] = 0

#     # replace langauge with binary tooo
#     # if lang in western powerhouses, replace with 0
    for i in range (len(lang_powerhouses)):
        combined_df.Language = combined_df.Language.apply(lambda x: '0' if lang_powerhouses[i] in x else x)

#     # if lang not in western, replace with 1
    filter  = (combined_df['Language'] != '0' )
    combined_df.loc[filter, 'Language'] = 1

#     # 0 value from string to int
    filter  = (combined_df['Language'] == '0' )
    combined_df.loc[filter, 'Language'] = 0

    return combined_df

imputed_df = imputation(combined_df)

Split into train/test, seperate data from target labels, fit to pipeline

In [12]:
print(imputed_df.head())
train_set, test_set = train_test_split(imputed_df, test_size= 0.3, random_state=28)
# # from now on only using train set -> df


df=train_set.drop(['Religion_AS', 'Religion_BU', 'Religion_CH', 'Religion_EA', 'Religion_IS', 'Religion_JU'],axis=1)
data_labels=train_set[['Religion_AS', 'Religion_BU', 'Religion_CH', 'Religion_EA', 'Religion_IS', 'Religion_JU']].copy()
# X_test = test_set.drop(['Religion_AS', 'Religion_BU', 'Religion_CH', 'Religion_EA', 'Religion_IS', 'Religion_JU'],axis=1 )
# y_test = test_set[['Religion_AS', 'Religion_BU', 'Religion_CH', 'Religion_EA', 'Religion_IS', 'Religion_JU']].copy()

# data_prepared = complete_pipeline.fit_transform(df)

df['Affiliations'] = df['Affiliations'].astype(int)
df['Language'] = df['Language'].astype(int)
df['Funding'] = df['Funding'].astype(int)

def fit():
  encoder.fit(df["Affiliations"].values.reshape(-1, 1))
  encoder.fit(df["Language"].values.reshape(-1, 1))
  encoder.fit(df["Funding"].values.reshape(-1, 1))
  # encoder.fit(df["Religion"].values.reshape(-1, 1))

def transform():
  encoder.transform(df["Affiliations"].values.reshape(-1, 1)).toarray()
  encoder.transform(df["Language"].values.reshape(-1, 1)).toarray()
  encoder.transform(df["Funding"].values.reshape(-1, 1)).toarray()
  # encoder.transform(df["Religion"].values.reshape(-1, 1)).toarray()

def fit_transform():
  fit()
  transform()

# # # seperate pipelines for numerical and cetegorical data
# # # num_attribs=["Year", "Cited by"] 
# # binary attributes into numeric timeline
# # think theres no categorical ones 
# num_attribs=["Cited"] 
# cat_attribs=list(df) 
# # # cat_attribs.remove("Year")
# cat_attribs.remove("Cited")


# scales values
complete_pipeline= Pipeline([ 
    # ('imputer',SimpleImputer(strategy="median")),
    ('std_scaler',StandardScaler())
])


data_prepared = complete_pipeline.fit_transform(df)

print(data_prepared.shape)
print(data_prepared)
print(data_labels)

# # numeric pipeline: scales values
# num_pipeline= Pipeline([
#     ('selector', DataFrameSelector(num_attribs)),
#     ('std_scaler',StandardScaler())
# ])

# # categorical pipeline: encodes 
# cat_pipeline = Pipeline([
#     ('selector',DataFrameSelector(cat_attribs)),
#     ('one hot',OneHotEncoder())
# ])


# full_pipeline = FeatureUnion(transformer_list=[
#     ("num_pipeline",num_pipeline),
#     ("cat_pipeline",cat_pipeline)
# ])


# data_prepared = full_pipeline.fit_transform(df)
# print(data_prepared)
# pd.set_option('display.max_rows', 200)





# print(df)

# # set feautures and predictors after fit_transform
# X = df[["Cited","Language", "Affiliations", "Funding"]]
# y = df[['Religion_BU', 'Religion_CH', 'Religion_AS', 'Religion_IS', 'Religion_JU', 'Religion_EA']]

# # training prepeared = x training targest = y
# # print(df.dtypes)
# print("printing x:")
# print(X)

   Cited Affiliations Funding Language  Religion_AS  Religion_BU  Religion_CH  \
0      0            0       1        0            0            1            0   
1      0            1       0        0            0            1            0   
2      0            0       0        0            0            1            0   
3      0            0       0        0            0            1            0   
4     10            0       0        0            0            1            0   

   Religion_EA  Religion_IS  Religion_JU  
0            0            0            0  
1            0            0            0  
2            0            0            0  
3            0            0            0  
4            0            0            0  
(17295, 4)
[[-0.26824161 -0.4692461  -0.34230964  4.22615835]
 [-0.31278115 -0.4692461  -0.34230964 -0.23662152]
 [-0.31278115 -0.4692461  -0.34230964 -0.23662152]
 ...
 [-0.09008344 -0.4692461  -0.34230964 -0.23662152]
 [ 1.06794464 -0.4692461  -0.342309

Linear regression with OLS

In [ ]:
# OLS
lin_reg=LinearRegression()
lin_reg.fit(data_prepared, data_labels)

# prepare unseen data 
some_data=df.iloc[:5]
some_labels=data_labels.iloc[:5]
some_data_prepared=complete_pipeline.transform(some_data)

# print results
print("OLS RESULTS")
print("Predictions:", lin_reg.predict(some_data_prepared))
print("Labels:", list(some_labels))
#mean squared errors
predictions=lin_reg.predict(data_prepared)
ols_mse = mean_squared_error(data_labels, predictions)
ols_rsme=np.sqrt(ols_mse)
print("OLS MSE: ", ols_mse)
print("OLS RSME:",ols_rsme)


OLS RESULTS
Predictions: [[0.07018651 0.20428408 0.26779825 0.2102953  0.11315652 0.13427934]
 [0.156327   0.16390618 0.166293   0.19901385 0.13916587 0.1752941 ]
 [0.156327   0.16390618 0.166293   0.19901385 0.13916587 0.1752941 ]
 [0.14221919 0.21474647 0.14354673 0.25956237 0.15053996 0.08938528]
 [0.22798842 0.12621947 0.13569459 0.20972961 0.22523664 0.07513127]]
Labels: ['Religion_AS', 'Religion_BU', 'Religion_CH', 'Religion_EA', 'Religion_IS', 'Religion_JU']
OLS MSE:  0.13729826557896604
OLS RSME: 0.3705378058700165


Multinomial Niave Bayes -> doesnt accept neg values, try random forrest! ... could also scale with minmaxscaler instead ...
Random Forest requires dense data, but data is sparse...

In [16]:
# classifier implementation:
# multiclass classification - Multinomial Naive Bayes
binary_rel_classifier = BinaryRelevance(RandomForestClassifier())
binary_rel_classifier.fit(data_prepared,data_labels)


# prepare unseen data 
some_data=df.iloc[:5]
some_labels=data_labels.iloc[:5]
some_data_prepared=complete_pipeline.transform(some_data)

# predictions = binary_rel_classifier.predict(X.values)
# print(accuracy_score(y,predictions))
# print(hamming_loss(y,predictions))

# print results
print("OLS RESULTS")
print("Predictions:", binary_rel_classifier.predict(some_data_prepared))
print("Labels:", list(some_labels))
#mean squared errors
predictions=binary_rel_classifier.predict(data_prepared)
ols_mse = mean_squared_error(data_labels, predictions)
ols_rsme=np.sqrt(ols_mse)
print("OLS MSE: ", ols_mse)
print("OLS RSME:",ols_rsme)

OLS RESULTS
Predictions: 
Labels: ['Religion_AS', 'Religion_BU', 'Religion_CH', 'Religion_EA', 'Religion_IS', 'Religion_JU']


TypeError: ignored

Support Vector Classification:

In [19]:
# new method - Support Vector Classification
binary_rel_classifier = BinaryRelevance(SVC())
binary_rel_classifier.fit(data_prepared,data_labels)

# prepare unseen data 
some_data=df.iloc[:5]
some_labels=data_labels.iloc[:5]
some_data_prepared=complete_pipeline.transform(some_data)

# predictions = binary_rel_classifier.predict(data_prepared)
# print(accuracy_score(data_labels,predictions))
# print(hamming_loss(data_labels,predictions))

# print results
print("OLS RESULTS")
print("Predictions:", binary_rel_classifier.predict(some_data_prepared))
print("Labels:", list(some_labels))
#mean squared errors
predictions=binary_rel_classifier.predict(data_prepared)
print(accuracy_score(data_labels,predictions))
print(hamming_loss(data_labels,predictions))
ols_mse = mean_squared_error(data_labels, predictions)
ols_rsme=np.sqrt(ols_mse)
print("OLS MSE: ", ols_mse)
print("OLS RSME:",ols_rsme)

OLS RESULTS
Predictions: 
Labels: ['Religion_AS', 'Religion_BU', 'Religion_CH', 'Religion_EA', 'Religion_IS', 'Religion_JU']
0.0006938421509106678
0.16656066300472197


TypeError: ignored

Linear SVC (made for sparse data)

In [21]:
binary_rel_classifier = BinaryRelevance(LinearSVC())
binary_rel_classifier.fit(data_prepared,data_labels)

# prepare unseen data 
some_data=df.iloc[:5]
some_labels=data_labels.iloc[:5]
some_data_prepared=complete_pipeline.transform(some_data)

# print results
print("OLS RESULTS")
print("Predictions:", binary_rel_classifier.predict(some_data_prepared))
print("Labels:", list(some_labels))
predictions = binary_rel_classifier.predict(data_prepared)
print(accuracy_score(data_labels,predictions))
print(hamming_loss(data_labels,predictions))

ols_mse = mean_squared_error(data_labels, predictions)
ols_rsme=np.sqrt(ols_mse)
print("OLS MSE: ", ols_mse)
print("OLS RSME:",ols_rsme)

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


OLS RESULTS
Predictions: 
Labels: ['Religion_AS', 'Religion_BU', 'Religion_CH', 'Religion_EA', 'Religion_IS', 'Religion_JU']
0.00017346053772766696
0.16670521345282838


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


TypeError: ignored

Grid Search:
recommends using Ridge/RidgeCV for better results?

In [23]:
print("LASSO RESULTS")
lasso= Lasso()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
grid=dict()
grid['alpha'] = np.linspace(0, 15, 1500)
search = GridSearchCV(lasso, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=1)
result = search.fit(data_prepared, data_labels)
print("MAE: %.3f" % result.best_score_)
print(result)
print(result.best_params_)
# grid search
# parameters = [
#     {
#         'classifier': [MultinomialNB()],
#         'classifier__alpha': [0.7, 1.0],
#     },
#     {
#         'classifier': [MultinomialNB()],
#         'classifier__alpha': [0.3, 1.5],
#     },
#     {
#         'classifier': [SVC()],
#         'classifier__kernel': ['rbf', 'linear'],
#     },
#     {
#         'classifier': [LinearSVC()],
#         'require_dense': [False, True],
#     }
# ]

# estimator.get_params().keys()

# svc = SVC(random_state=0)

# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=0)

# search = GridSearchCV(estimator=svc, param_grid=param_grid, scoring="roc_auc", cv=cv)

# clf = GridSearchCV(BinaryRelevance(), parameters, scoring='accuracy', error_score='raise')
# # # # # cv=cv throws error that data is still multilabel not multiclass
# clf.fit(X.values, y)

# param_grid  = [{'n_estimators': [3, 10, 30],'max_features':[2, 4, 6, 8]}, 
#                {'bootstrap': [False],'n_estimators':[3,10],'max_features':[2,3,4]}]

# svc=MultinomialNB()

# grid_search = GridSearchCV(svc, param_grid, cv=5,scoring='neg_mean_squared_error')
# grid_search.fit(X, y)

# predictions = clf.predict(X.values)
# print(accuracy_score(y,predictions))
# print(hamming_loss(y,predictions))

# print(clf.best_params_, clf.best_score_)

# best_classifier=clf.best_estimator_

LASSO RESULTS


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:686: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.058e+03, tolerance: 2.136e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.e

MAE: -0.275
GridSearchCV(cv=RepeatedKFold(n_repeats=3, n_splits=10, random_state=1),
             estimator=Lasso(), n_jobs=1,
             param_grid={'alpha': array([0.00000000e+00, 1.00066711e-02, 2.00133422e-02, ...,
       1.49799867e+01, 1.49899933e+01, 1.50000000e+01])},
             scoring='neg_mean_absolute_error')
{'alpha': 0.0}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.155e+03, tolerance: 2.335e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.017e+03, tolerance: 2.079e-01 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_mode